In [108]:
import pandas as pd
import numpy as np
import os
from ast import literal_eval

pd.options.mode.chained_assignment = None

In [109]:
data_path = "./data"
if os.listdir(data_path) != ['tmdb_5000_credits.csv', 'tmdb_5000_movies.csv']:
    print("[ERROR] Please download and unzip the dataset in a subdirectory './data'.")
else:
    print("[INFO] The dataset is correctly placed.")

[INFO] The dataset is correctly placed.


### Loading the data

In [185]:
tmdb_credits = pd.read_csv(os.path.join(data_path, "tmdb_5000_credits.csv"))
tmdb_credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [186]:
tmdb_movies = pd.read_csv(os.path.join(data_path, "tmdb_5000_movies.csv"))
tmdb_movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [187]:
def parse_genres(genres):
    L=[]
    for genre in genres:
        L.append(genre['name'])
    if len(L) == 0:
        return np.nan
    return L
def parse_comp(comps):
    L=[]
    for comp in comps:
        L.append(comp['name'])
    if len(L) == 0:
        return np.nan
    return L

In [188]:
pc = tmdb_movies[["production_companies", "release_date"]]
pc.head()

,production_companies,release_date
0,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",2009-12-10
1,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",2007-05-19
2,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",2015-10-26
3,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",2012-07-16
4,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",2012-03-07


In [189]:
pc["production_companies"] = pc["production_companies"] .apply(lambda x: parse_comp(literal_eval(x)))
pc.head()

,production_companies,release_date
0,"[Ingenious Film Partners, Twentieth Century Fo...",2009-12-10
1,"[Walt Disney Pictures, Jerry Bruckheimer Films...",2007-05-19
2,"[Columbia Pictures, Danjaq, B24]",2015-10-26
3,"[Legendary Pictures, Warner Bros., DC Entertai...",2012-07-16
4,[Walt Disney Pictures],2012-03-07


In [190]:
pc = pc.dropna().explode("production_companies").reset_index().drop(columns="index")
pc.head()

,production_companies,release_date
0,Ingenious Film Partners,2009-12-10
1,Twentieth Century Fox Film Corporation,2009-12-10
2,Dune Entertainment,2009-12-10
3,Lightstorm Entertainment,2009-12-10
4,Walt Disney Pictures,2007-05-19


In [191]:
top_pc = pc.groupby("production_companies").count().sort_values("release_date",ascending=False )
top_pc_list = list(top_pc.head(10).reset_index()["production_companies"])
top_pc_list
#top_pc

['Warner Bros.',
 'Universal Pictures',
 'Paramount Pictures',
 'Twentieth Century Fox Film Corporation',
 'Columbia Pictures',
 'New Line Cinema',
 'Metro-Goldwyn-Mayer (MGM)',
 'Touchstone Pictures',
 'Walt Disney Pictures',
 'Relativity Media']

In [192]:
pc = pc[pc["production_companies"].isin(top_pc_list)]
pc.reset_index(inplace=True)
pc.drop(columns=["index"], inplace=True)
pc

,production_companies,release_date
0,Twentieth Century Fox Film Corporation,2009-12-10
1,Walt Disney Pictures,2007-05-19
2,Columbia Pictures,2015-10-26
3,Warner Bros.,2012-07-16
4,Walt Disney Pictures,2012-03-07
...,...,...
1954,Warner Bros.,1989-09-01
1955,New Line Cinema,2015-07-10
1956,Twentieth Century Fox Film Corporation,1998-01-01
1957,Metro-Goldwyn-Mayer (MGM),1983-03-24


In [193]:
pc['release_date'] = pd.to_datetime(pc['release_date'])
pc['year'] = pc['release_date'].apply(lambda x: x.strftime('%Y'))
pc.head()

,production_companies,release_date,year
0,Twentieth Century Fox Film Corporation,2009-12-10,2009
1,Walt Disney Pictures,2007-05-19,2007
2,Columbia Pictures,2015-10-26,2015
3,Warner Bros.,2012-07-16,2012
4,Walt Disney Pictures,2012-03-07,2012


In [194]:
pc = pc[pc["year"].astype(int)>1965]
pc = pc[pc["year"].astype(int)<2017]

In [195]:
df = pc.groupby(["year","production_companies"],  as_index=False).count()
df.head()

,year,production_companies,release_date
0,1966,Twentieth Century Fox Film Corporation,1
1,1966,Universal Pictures,1
2,1966,Warner Bros.,1
3,1967,Metro-Goldwyn-Mayer (MGM),1
4,1968,Metro-Goldwyn-Mayer (MGM),1


In [196]:
df['norm_per_year'] = df['release_date'] / df.groupby('year')['release_date'].transform('sum')
df.head(40)

,year,production_companies,release_date,norm_per_year
0,1966,Twentieth Century Fox Film Corporation,1,0.333333
1,1966,Universal Pictures,1,0.333333
2,1966,Warner Bros.,1,0.333333
3,1967,Metro-Goldwyn-Mayer (MGM),1,1.000000
4,1968,Metro-Goldwyn-Mayer (MGM),1,0.250000
5,1968,Paramount Pictures,3,0.750000
6,1969,Paramount Pictures,1,0.250000
7,1969,Twentieth Century Fox Film Corporation,1,0.250000
8,1969,Universal Pictures,2,0.500000
9,1970,Paramount Pictures,5,0.454545


In [197]:
years = list(pc.year.unique())
#years.append("2017")
#years.append("1965")
#a.insert(len(a),5)
pcs = list(pc.production_companies.unique())

index = pd.MultiIndex.from_product([years,pcs], names = ["year", "production_companies"])

zero_df = pd.DataFrame(index = index).reset_index()
zero_df["release_date"] = 0
zero_df["norm_per_year"] = 0.00

In [198]:
zero_df.shape

(510, 4)

In [199]:
df = pd.concat([df,zero_df])

In [200]:
df.drop_duplicates(subset=['year', 'production_companies'], inplace=True, keep='first')

In [201]:
a = df[df.year=="1966"].copy()
a.year = "1965"
b = df[df.year=="2016"].copy()
b.year = "2017"
df = pd.concat([a,df])
df = pd.concat([df,b])

In [203]:
df.rename(columns={"release_date": "total_films"}, inplace=True)
df.head()

,year,production_companies,total_films,norm_per_year
0,1965,Twentieth Century Fox Film Corporation,1,0.333333
1,1965,Universal Pictures,1,0.333333
2,1965,Warner Bros.,1,0.333333
491,1965,Walt Disney Pictures,0,0.000000
492,1965,Columbia Pictures,0,0.000000


In [204]:
df.groupby("production_companies").count().shape

(10, 3)

In [163]:
#df[df.norm_per_year==0]["norm_per_year"] = 0.1

In [205]:
df.head()

,year,production_companies,total_films,norm_per_year
0,1965,Twentieth Century Fox Film Corporation,1,0.333333
1,1965,Universal Pictures,1,0.333333
2,1965,Warner Bros.,1,0.333333
491,1965,Walt Disney Pictures,0,0.000000
492,1965,Columbia Pictures,0,0.000000


In [206]:
df.sort_values(["year","production_companies"],inplace=True)

In [207]:
df.head(30)

,year,production_companies,total_films,norm_per_year
492,1965,Columbia Pictures,0,0.000000
496,1965,Metro-Goldwyn-Mayer (MGM),0,0.000000
495,1965,New Line Cinema,0,0.000000
494,1965,Paramount Pictures,0,0.000000
498,1965,Relativity Media,0,0.000000
499,1965,Touchstone Pictures,0,0.000000
0,1965,Twentieth Century Fox Film Corporation,1,0.333333
1,1965,Universal Pictures,1,0.333333
491,1965,Walt Disney Pictures,0,0.000000
2,1965,Warner Bros.,1,0.333333


In [209]:
df.to_csv("production_companies_first_level_stream.csv")

#### Genres

In [210]:
pc = tmdb_movies[["genres", "release_date", "production_companies"]]
pc.head()

,genres,release_date,production_companies
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2009-12-10,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",2007-05-19,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2015-10-26,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",2012-07-16,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2012-03-07,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"


In [211]:
def parse_comp(comps):
    L=[]
    for comp in comps:
        L.append(comp['name'])
    if len(L) == 0:
        return np.nan
    return L


In [212]:
pc.head()

,genres,release_date,production_companies
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2009-12-10,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",2007-05-19,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2015-10-26,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",2012-07-16,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2012-03-07,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"


In [213]:
pc["genres"] = pc.genres.apply(lambda x: parse_genres(literal_eval(x)))
pc.head()

,genres,release_date,production_companies
0,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,"[Adventure, Fantasy, Action]",2007-05-19,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,"[Action, Adventure, Crime]",2015-10-26,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,"[Action, Crime, Drama, Thriller]",2012-07-16,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,"[Action, Adventure, Science Fiction]",2012-03-07,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"


In [214]:
pc["production_companies"] = pc["production_companies"] .apply(lambda x: parse_comp(literal_eval(x)))
pc.head()

,genres,release_date,production_companies
0,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,"[Ingenious Film Partners, Twentieth Century Fo..."
1,"[Adventure, Fantasy, Action]",2007-05-19,"[Walt Disney Pictures, Jerry Bruckheimer Films..."
2,"[Action, Adventure, Crime]",2015-10-26,"[Columbia Pictures, Danjaq, B24]"
3,"[Action, Crime, Drama, Thriller]",2012-07-16,"[Legendary Pictures, Warner Bros., DC Entertai..."
4,"[Action, Adventure, Science Fiction]",2012-03-07,[Walt Disney Pictures]


In [215]:
pc = pc.dropna().explode("production_companies").reset_index().drop(columns="index")
pc.head()

,genres,release_date,production_companies
0,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,Ingenious Film Partners
1,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,Twentieth Century Fox Film Corporation
2,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,Dune Entertainment
3,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,Lightstorm Entertainment
4,"[Adventure, Fantasy, Action]",2007-05-19,Walt Disney Pictures


In [216]:
top_pc = pc.groupby("production_companies").count().sort_values("release_date",ascending=False )
top_pc_list = list(top_pc.head(10).reset_index()["production_companies"])
top_pc_list

['Warner Bros.',
 'Universal Pictures',
 'Paramount Pictures',
 'Twentieth Century Fox Film Corporation',
 'Columbia Pictures',
 'New Line Cinema',
 'Metro-Goldwyn-Mayer (MGM)',
 'Touchstone Pictures',
 'Walt Disney Pictures',
 'Relativity Media']

In [217]:
pc = pc.dropna().explode("genres").reset_index().drop(columns="index")
pc.head()

,genres,release_date,production_companies
0,Action,2009-12-10,Ingenious Film Partners
1,Adventure,2009-12-10,Ingenious Film Partners
2,Fantasy,2009-12-10,Ingenious Film Partners
3,Science Fiction,2009-12-10,Ingenious Film Partners
4,Action,2009-12-10,Twentieth Century Fox Film Corporation


In [218]:
pc = pc[pc["production_companies"].isin(top_pc_list)]
pc.reset_index(inplace=True)
pc.drop(columns=["index"], inplace=True)
pc

,genres,release_date,production_companies
0,Action,2009-12-10,Twentieth Century Fox Film Corporation
1,Adventure,2009-12-10,Twentieth Century Fox Film Corporation
2,Fantasy,2009-12-10,Twentieth Century Fox Film Corporation
3,Science Fiction,2009-12-10,Twentieth Century Fox Film Corporation
4,Adventure,2007-05-19,Walt Disney Pictures
...,...,...,...
5283,Drama,1983-03-24,Metro-Goldwyn-Mayer (MGM)
5284,Romance,1983-03-24,Metro-Goldwyn-Mayer (MGM)
5285,Action,1992-09-04,Columbia Pictures
5286,Crime,1992-09-04,Columbia Pictures


In [219]:
pc['release_date'] = pd.to_datetime(pc['release_date'])
pc['year'] = pc['release_date'].apply(lambda x: x.strftime('%Y'))
pc.head()

,genres,release_date,production_companies,year
0,Action,2009-12-10,Twentieth Century Fox Film Corporation,2009
1,Adventure,2009-12-10,Twentieth Century Fox Film Corporation,2009
2,Fantasy,2009-12-10,Twentieth Century Fox Film Corporation,2009
3,Science Fiction,2009-12-10,Twentieth Century Fox Film Corporation,2009
4,Adventure,2007-05-19,Walt Disney Pictures,2007


In [220]:

genres_to_drop = ["TV Movie","Foreign", "Mystery", "Documentary", "Animation"]
pc = pc[pc["year"].astype(int)>1965]
pc = pc[pc["year"].astype(int)<2017]
pc = pc[~pc["genres"].isin(genres_to_drop)]


In [221]:
df2 = pc.groupby(["year","production_companies","genres"],  as_index=False).count()
df2.head()

,year,production_companies,genres,release_date
0,1966,Twentieth Century Fox Film Corporation,Adventure,1
1,1966,Twentieth Century Fox Film Corporation,Comedy,1
2,1966,Twentieth Century Fox Film Corporation,Crime,1
3,1966,Twentieth Century Fox Film Corporation,Family,1
4,1966,Twentieth Century Fox Film Corporation,Science Fiction,1


In [222]:
df2['norm_per_year'] = df2['release_date'] / df2.groupby(['year','production_companies'])['release_date'].transform('sum')
df2.head()

,year,production_companies,genres,release_date,norm_per_year
0,1966,Twentieth Century Fox Film Corporation,Adventure,1,0.2
1,1966,Twentieth Century Fox Film Corporation,Comedy,1,0.2
2,1966,Twentieth Century Fox Film Corporation,Crime,1,0.2
3,1966,Twentieth Century Fox Film Corporation,Family,1,0.2
4,1966,Twentieth Century Fox Film Corporation,Science Fiction,1,0.2


In [223]:
years = list(pc.year.unique())
gens = list(pc.genres.unique())
comps = list(pc.production_companies.unique())

index = pd.MultiIndex.from_product([years, gens, comps], names = ["year", "genres", "production_companies"])

zero_df = pd.DataFrame(index = index).reset_index()
zero_df["release_date"] = 0
zero_df["norm_per_year"] = 0.0

In [224]:
zero_df

,year,genres,production_companies,release_date,norm_per_year
0,2009,Action,Twentieth Century Fox Film Corporation,0,0.0
1,2009,Action,Walt Disney Pictures,0,0.0
2,2009,Action,Columbia Pictures,0,0.0
3,2009,Action,Warner Bros.,0,0.0
4,2009,Action,Paramount Pictures,0,0.0
...,...,...,...,...,...
7645,1967,Music,New Line Cinema,0,0.0
7646,1967,Music,Metro-Goldwyn-Mayer (MGM),0,0.0
7647,1967,Music,Universal Pictures,0,0.0
7648,1967,Music,Relativity Media,0,0.0


In [225]:
# We must have all genres in all years, so we populate 0 where missing
df3 = pd.concat([df2,zero_df])
df3 = df3.drop_duplicates(subset=['year', 'genres','production_companies'], keep='first')

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [226]:
df3.shape

(7650, 5)

In [229]:
a = df3[df3.year=="1966"].copy()
a.year = "1965"
b = df3[df3.year=="2016"].copy()
b.year = "2017"
df3 = pd.concat([a,df3])
df3 = pd.concat([df3,b])

In [230]:
df3.rename(columns={"release_date": "total_films"}, inplace=True)
df3.head()

,genres,norm_per_year,production_companies,total_films,year
0,Adventure,0.2,Twentieth Century Fox Film Corporation,1,1965
1,Comedy,0.2,Twentieth Century Fox Film Corporation,1,1965
2,Crime,0.2,Twentieth Century Fox Film Corporation,1,1965
3,Family,0.2,Twentieth Century Fox Film Corporation,1,1965
4,Science Fiction,0.2,Twentieth Century Fox Film Corporation,1,1965


In [231]:
df3.sort_values(["year","production_companies", "genres"],inplace=True)
df3.head()

,genres,norm_per_year,production_companies,total_films,year
7352,Action,0.0,Columbia Pictures,0,1965
7362,Adventure,0.0,Columbia Pictures,0,1965
7452,Comedy,0.0,Columbia Pictures,0,1965
7392,Crime,0.0,Columbia Pictures,0,1965
7402,Drama,0.0,Columbia Pictures,0,1965


In [232]:
df3.to_csv("genres_second_level_stream.csv")